# **Práctica 2**

En esta práctica vamos a ver como podríamos trabajar con redes neuronales totalmente conectadas en imagen. Veremos cuales son las limitaciones de emplear este tipo de arquitecturas para trabajar con imágenes.


En primer lugar vamos a trabajar con el dataset MNIST para clasificación de imágenes de dígitos numéricos. Para ello seguiremos los siguientes pasos:



1.   Descargaremos las imágenes y las visualizaremos
2.   Pre-procesaremos los datos.
3.   Diseñaremos la arquitectura.
4.   Entrenaremos la red.
5.   Evaluaremos el modelo entrenado.



# 1. Descaga de las imágenes y visualización

In [ ]:
# Importamos la base de datos de las propias de keras
from tensorflow.keras.datasets import mnist

# Descargamos el dataset diferenciando entre conjunto de entrenamiento y validación/test
(X_train, y_train), (X_testval, y_testval) = mnist.load_data()

In [ ]:
# Analizamos la base de datos descargada
# Tamaño de las imágenes y número de muestras
print("Tamaño imágenes entrenamiento: ", X_train.shape)
print("Tamaño etiquetas entrenamiento: ", y_train.shape)
print("Tamaño imágenes validación/test: ", X_testval.shape)
print("Tamaño etiquetas validación/test: ", y_testval.shape)

In [ ]:
# Tipo de datos
print("Tipo datos imágenes entrenamiento: ", X_train.dtype)
print("Tipo datos etiquetas entrenamiento: ", y_train.dtype)
print("Tipo datos imágenes validación/test: ", X_testval.dtype)
print("Tipo datos etiquetas validación/test: ", y_testval.dtype)

In [ ]:
# Rango de valores de las imágenes
print("Valor mínimo imágenes entrenamiento: ", X_train.min())
print("Valor máximo imágenes entrenamiento: ", X_train.max())
print("Valor mínimo imágenes validación/test: ", X_testval.min())
print("Valor máximo imágenes validación/test: ", X_testval.max())

In [ ]:
# Etiquetas de las particiones
import numpy as np

print("Etiquetas entrenamiento: ", np.unique(y_train))
print("Etiquetas validación/test: ", np.unique(y_testval))

In [ ]:
# Visualización de las imágenes
import matplotlib.pyplot as plt
# Configuramos tamaño de las imágenes para una correcta visualización
plt.rcParams['figure.figsize'] = (10, 10)
# Mostramos 9 imágenes con su etiqueta correspondiente
for i in range(9):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_train[i], cmap='gray')
  plt.title(f'Clase {y_train[i]}')

# 2. Pre-proceso de los datos

In [ ]:
# Para trabajar con redes neuronales totalmente conectadas no podemos trabajar con matrices 2D, debemos convertir la imagen a vector 1D
X_train_vector = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
X_testval_vector = X_testval.reshape(X_testval.shape[0], X_testval.shape[1]*X_testval.shape[2])
print("Nuevo tamaño datos entrenamiento: ", X_train_vector.shape)
print("Nuevo tamaño datos validación/test: ", X_testval_vector.shape)

In [ ]:
# Las redes neuronales trabajan mejor con valores entre 0-1. Por lo que vamos a convertir el rango

# Primero convertimos las imágenes a float
X_train_vector = X_train_vector.astype('float32')
X_testval_vector = X_testval_vector.astype('float32')
print('Tipo datos entrenamiento: ', X_train_vector.dtype)
print('Tipo datos validación/test: ', X_testval_vector.dtype)

In [ ]:
# Cambiamos rango de las imágenes
X_train_vector /= 255
X_testval_vector /= 255
print("Rango datos entrenamiento: [", X_train_vector.min(), ', ', X_train_vector.max(), ']')
print("Rango datos validación/test: [", X_testval_vector.min(), ', ', X_testval_vector.max(), ']')

In [10]:
# Convertimos etiquetas a codificación one-hot
from tensorflow.keras.utils import to_categorical
num_clases = len(np.unique(y_train))
y_train_cod = to_categorical(y_train, num_clases)
y_testval_cod = to_categorical(y_testval, num_clases)

In [ ]:
print("Tamaño etiquetas entrenamiento: ", y_train_cod.shape)
print("Tamaño etiquetas validación/test: ", y_testval_cod.shape)

In [ ]:
# Dividimos conjunto de datos de validación/test en 2 subconjuntos
samples_test_nb = int(X_testval.shape[0]/2)
X_val = X_testval_vector[:samples_test_nb]
y_val = y_testval_cod[:samples_test_nb]
X_test = X_testval_vector[samples_test_nb:]
y_test = y_testval_cod[samples_test_nb:]

print("Muestras validación: ", X_val.shape)
print("Muestras test: ", X_test.shape)

# 3. Diseñamos la arquitectura


In [13]:
# Importamos dependencias
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

In [ ]:
# Definimos la arquitectura
# Definimos arquitectura
input_layer = Input(shape=(X_train_vector.shape[1],))
hidden_layer = Dense(32, activation="relu")(input_layer)
output_layer = Dense(num_clases, activation="softmax")(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.summary()

In [15]:
# Compilamos el modelo
model.compile(loss="categorical_crossentropy", optimizer="adam",
              metrics=["accuracy"])

# 4. Entrenamiento de un modelo

In [ ]:
history = model.fit(X_train_vector, y_train_cod, epochs=20, batch_size=128,
                    validation_data=(X_val, y_val))

Cosas a observar:

Métricas entrenamiento
Métricas validacón
¿Sobreajuste?

In [ ]:
# Visualizamos la precisión
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.ylim(0,1)
plt.legend(['Entrenamiento', 'Validación'], loc="lower right")
plt.show()

In [ ]:
# Visualizamos pérdidas
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Pérdidas modelo')
plt.ylabel('Pérdidas')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Validación'], loc="upper left")
plt.show()

In [19]:
# Guardamos el modelo
from pathlib import Path
path_modelos = Path('./modelos')
path_modelos.mkdir(exist_ok=True)
model.save(path_modelos / 'model_mnist.h5')

# 5. Evaluamos el modelo

In [ ]:
# Sacamos métricas sobre nuestro conjunto de test
metrics = model.evaluate(X_test, y_test, verbose=0)
print("Precision test: ", metrics[1])

In [21]:
# Obtenemos predicciones 
prediccion = model.predict(X_test)
# Cogemos la clase con mayor probabilidad
prediccion = np.argmax(prediccion, axis=1)

In [22]:
# Deshacemos codificación one-hot en conjunto de test
y_test_clases = np.argmax(y_test, axis=1)

In [23]:
# Detectamos imágenes correctamente clasificadas
correct_index = np.nonzero(prediccion == y_test_clases)[0]

In [24]:
# Detectamos imágenes incorrectamente clasificadas
incorrect_index = np.nonzero(prediccion != y_test_clases)[0]

In [ ]:
# Mostramos imágenes correctamente clasificadas
plt.figure()
for i, correct in enumerate(correct_index[:9]):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_test[correct].reshape(28,28), cmap='gray')
  plt.title(f'Real {y_test_clases[correct]}, Predicha {prediccion[correct]}')

In [ ]:
# Mostramos imágenes incorrectamente clasificadas
plt.figure()
for i, incorrect in enumerate(incorrect_index[:9]):
  plt.subplot(3, 3, i+1)
  plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray')
  plt.title(f'Real {y_test_clases[incorrect]}, Predicha {prediccion[incorrect]}')

# Ejercicio 1: Probamos diferentes funciones de activación

Analizamos cómo afecta el uso de diferentes funciones de activación en la capa oculta. Probamos las siguientes:

*   Sin función de activación
*   Sigmoid
*   Tanh
*   Relu




# Ejercicio 2: Creamos un modelo más complejo
Para aumentar la complejidad del modelo añadimos una capa oculta de 512 neuronas

# Ejercicio 3: Entrenamos un modelo con la base de datos CIFAR10